In [1]:
from sympy import *
import sympy
from scipy.integrate import quad
from scipy.optimize import minimize
x,a = symbols('x a')
import math

In [2]:
rho = 0.5
## define CES production function and its derivative
def g(p, rho = rho, alpha = 0.15):
    if rho == 0:
        return alpha**(-alpha) * (1-alpha)**(-(1-alpha)) * p**alpha
    else:
        t1 = (1-alpha)**(1/(1-rho))
        t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
        return (t1 + t2)**(-(1-rho)/rho)

def gprime_new(p, rho = rho, alpha = 0.15):
    if rho == 0:
        return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))
    else:
        t1 = (1-alpha)**(1/(1-rho))
        t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
        coef = alpha **(1/(1-rho)) * p**(-rho/(1-rho) - 1)
        return (t1 + t2)**(-(1-rho)/rho - 1) * coef
    
def gprime(p, rho = rho, alpha = 0.15):
    if rho == 0:
        return (alpha/(1-alpha))**(1-alpha) * p**(-(1-alpha))
    else:
        res = diff(g(x, rho, alpha),x).subs(x,p)
        if type(res) == sympy.core.numbers.Float:
            res = float(res)
        return res
    
def obj_fun(alpha):
    return abs(gprime(1,alpha) / g(1,alpha) - 0.15)

In [3]:
gprime(1, alpha = 0.2)

0.08650519031141866

In [4]:
solve(gprime_new(1,rho, a) / g(1,rho, a) - 0.15 , a)

[-0.724387744895918, 0.295816316324489]

In [13]:
solve(gprime(1,rho, a) / g(1,rho, a) - 0.15 , a)

[0.0690161379156130]

In [5]:
alpha_p = 0.295816316324489
gprime(1,rho, alpha_p) / g(1,rho, alpha_p)

0.14999999999999966

In [20]:
gprime_new(x,rho, alpha_p)

0.176470588235294*(0.176470588235294*x**0.99009900990099 + 1.0)**0.01/x**0.00990099009900991

In [23]:
(alpha_p) **(1/(1-rho))

0.1764705882352941

In [5]:
def jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe) + tb * gprime(pe))**(-theta) * Ceystar
    return num / denum

def j0_new(pe,tb,jxbar,theta,Cex,Ceystar):
    num = g(pe + tb)**(-theta) * Cex
    denum = g(pe+tb)**(-theta) * Cex + (g(pe))**(-theta) * Ceystar
    return num / denum

def Ceystar_new(pe,tb, jxbar_p, jxbar, sigmastar, theta, Ceystar):
    const = gprime(pe) * g(pe)**(-sigmastar) * Ceystar
    frac = ((1-jxbar_p)/ (1-jxbar))**(1+(1-sigmastar)/theta)
    #frac = ((1-jxbar_new(pe,tb,jxbar,theta,Cex,Ceystar))/ (1-jxbar))**(1+(1-sigmastar)/theta)
    return const * frac / (g(1)**(-sigmastar) * gprime(1))

def Cey_new(pe,tb, sigmastar, theta, Cey):
    const = gprime(pe + tb) * g(pe+tb)**(-sigma) * Cey
    return const / (g(1)**(-sigma) * gprime(1))

def Cex1_new(pe,tb, j0bar_p, j0bar, sigmastar, Cex):
    const = gprime(pe + tb) * g(pe+tb)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = (j0bar_p / j0bar) ** (1+ (1-sigmastar)/theta)
    return const * frac

def Cex2_new(pe,tb,jxbar_p, j0bar_p, jxbar, sigmastar, theta):
    const = g(pe)**(-sigmastar) * gprime(pe + tb) / (g(1)**(-sigmastar) * gprime(1)) * Cex
    frac = ((1-jxbar)/jxbar)**(sigmastar/theta) * (theta + 1 - sigmastar)/theta
    jterm = 1/ jxbar**(1+(1-sigmastar)/theta)
    B1, B2 = imcomp_betas(j0bar_p, jxbar_p, theta, sigmastar)
    return const * frac * jterm * (B2- B1)
    

def Cem_new(pe, tb, sigma, Cem):
    const = gprime(pe + tb) * g(pe + tb)**(-sigma) / (g(1)**(-sigmastar) * gprime(1)) * Cem
    return const
    
def Vgx2_new(pe, jxbar_p, jxbar, j0bar_p, Cex, theta, sigmastar):
    pterm = g(pe)**(1-sigmastar) *g(1)* Cex / (gprime(1) * g(1)**(1-sigmastar))
    num = (1-j0bar_p)**((theta + 1 - sigmastar)/theta) - (1-jxbar_p)**((theta + 1 - sigmastar) / theta)
    denum = jxbar * (1-jxbar)**((1-sigmastar)/theta)
    return pterm * num / denum

def Vgx1_new(pe, tb, j0_p, jxbar, Cex, theta, sigmastar):
    const = (g(pe+tb) / g(1))**(1-sigmastar) * (j0_p / jxbar)**(1+(1-sigmastar)/theta) * g(1)/gprime(1) * Cex
    return const

def S_new(pe, tb, Cex2_prime, Vgx2_prime):
    const = g(pe+tb) / gprime(pe+tb) * Cex2_prime - Vgx2_prime
    return const

def Vgm_new(pe, tb, Cem):
    return g(pe+tb)**(1-sigma) * Cem * g(1) / (g(1)**(1-sigma) * gprime(1))

def Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime):
    return (1/k(pe+tb)) * (Cey_prime + Cex1_prime + Cex2_prime)

def Lgstar_new(pe,tb,Ceystar_prime, Cem_prime):
    return 1/k(pe) * Ceystar_prime + 1/k(pe+tb) * Cem_prime

def Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime):
    Vgystar_prime = (g(pe) / g(1))**(1-sigmastar) * ((1-jxbar_prime)/(1-jxbar)) ** (1+(1-sigmastar)/theta) * g(1)/ gprime(1) * Ceystar
    return Vgystar_prime + Vgx1_prime + Vgx2_prime


In [6]:
jxbar_prime = jxbar_new(pe, tb, jxbar, theta, Cex, Ceystar)
jxbar_prime = jxbar
j0_prime = j0_new(pe,tb,jxbar,theta,Cex,Ceystar)
Cey_prime = Cey_new(pe,tb, sigmastar, theta, Cey)
Cex1_prime = Cex1_new(pe,tb, j0_prime, jxbar, sigmastar, Cex)
Cex2_prime = Cex2_new(pe, tb, jxbar_prime, j0_prime, jxbar, sigmastar, theta)
Cem_prime = Cem_new(pe, tb, sigma, Cem)
Ceystar_prime = Ceystar_new(pe,tb, jxbar_prime, jxbar, sigmastar, theta, Ceystar)
Vgx1_prime = Vgx1_new(pe, tb, j0_prime, jxbar, Cex, theta, sigmastar)
Vgx2_prime = Vgx2_new(pe, jxbar_prime, jxbar, j0_prime, Cex, theta, sigmastar)
Vgm_prime = Vgm_new(pe, tb, Cem)
S = S_new(pe, tb, Cex2_prime, Vgx2_prime)
Lg_prime = Lg_new(pe, tb, Cey_prime, Cex1_prime, Cex2_prime)
Lgstar_prime =Lgstar_new(pe,tb,Ceystar_prime, Cem_prime)
Vgstar_prime = Vgstar_new(pe,jxbar_prime,jxbar, sigmastar,theta,Ceystar, Vgx1_prime, Vgx2_prime)

In [7]:
Cex_prime = gprime(pe) * g(pe)**(-sigmastar) / (g(1)**(-sigmastar) * gprime(1)) * (jxbar_prime / jxbar) ** (1+ (1-sigmastar)/theta) * Cex
Ce_prime = Cey_prime + Cem_prime
Cestar_prime = Ceystar_prime + Cex_prime
Ceworld_prime = Ce_prime + Cestar_prime
Qe_prime =4.40505495
Qestar_prime = 27.33067971
Qeworld_prime = Qe_prime + Qestar_prime


In [8]:
dcestardpe = abs(Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * Cex_prime 
                      #+ D1starprime(pe,tb,sigmastar) / D1star(pe,tb,sigmastar) * Cex2_prime
                      + Dstarprime(pe,sigmastar) / Dstar(pe,sigmastar) * (Ceystar_prime))
numerator = varphi * (epsilonS * Qe_prime + epsilonSstar * Qestar_prime)
denominator = numerator / varphi + dcestardpe * pe

In [9]:
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
numerator = varphi * epsilonSw * Qeworld_prime
denominator = epsilonSw * Qeworld_prime + epsilonDstar * Cestar_prime

In [10]:
rho = 0.0001

In [11]:
def g1(p, rho = rho):
    alpha = 0.15
    t1 = (1-alpha)**(1/(1-rho))
    t2 = alpha**(1/(1-rho)) * p**(-rho/(1-rho))
    return (t1 + t2)**(-(1-rho)/rho)

def g1prime(p, rho = rho):
    alpha = 0.15
    return diff(g1(x, rho),x).subs(x,p)

In [196]:
pe = 0.965289922
tb = 0.368354551
prop = 0.449691508
re = (pe + tb) / pe
ve = (pe + tb - prop*tb)
jmbar_prime =0.767046309
jxbar_prime = 0.014224486
Cey_prime = 3.447997017
Cex_prime = 0.32250493
Cem_prime = 0.896873797
Ceystar_prime = 27.04347061
Qe_prime = 4.40160679
Qestar_prime = 27.30928598
epsilonSw = (Qe_prime) * epsilonS / Qeworld_prime + Qestar_prime * epsilonSstar / Qeworld_prime
djxbardpe = theta * gprime(pe) / g(pe) * jxbar_prime * (1-jxbar_prime)
djxbardpe = diff(jxbar_temp(x,x+tb-prop*tb,theta,Cex,Ceystar),x).subs(x,pe)

In [197]:
## new leakage for PC, EPC
djxdve = -jxbar_prime * (1-jxbar_prime) * gprime(ve) / g(ve) * theta
djmdve = -jmbar_prime * (1-jmbar_prime) * gprime(ve) / g(ve) * theta

dceydve = Dstarprime(ve, sigma) / Dstar(ve, sigma) * Cey_prime + (1+(1-sigma)/theta) * djmdve / jmbar_prime * Cey_prime
dcemdve = 1/(1-jmbar_prime) * (1+(1-sigma)/theta) * (-djmdve) * Cem_prime 
dcexdve = Dstarprime(ve, sigmastar) / Dstar(ve, sigmastar) * Cex_prime +  (1+(1-sigmastar) / theta) * Cex_prime / jxbar_prime * djxdve
dceystardve = (1+(1-sigmastar) /theta) * Ceystar_prime * (-djxdve) / (1-jxbar_prime)

leak = -(dceystardve + dcemdve) / (dcexdve + dceydve)
leak2 = -dceystardve / dcexdve

In [213]:
numerator = varphi * epsilonSw * Qeworld_prime
dceystardpe = abs(Dstarprime(pe, sigmastar) / Dstar(pe, sigmastar) - (1+(1-sigmastar) / theta) / (1-jxbar_prime) * djxbardpe) * Ceystar_prime
dcexdpe = abs(Dstarprime(ve, sigmastar) / Dstar(ve, sigmastar)+(1+(1-sigmastar) / theta) / (jxbar_prime) * djxbardpe) * Cex_prime

In [215]:
dceystardpe * pe+ leak2 * dcexdpe * pe

27.1884990213406

In [165]:
epsilonDstar * Ceystar_prime + prop * epsilonDstar * Cex_prime

27.188498338309135

In [203]:
dceystardpe * pe

27.0835277785971

In [140]:
leak2 * epsilonDstar * Cex_prime

0.145028408870602

In [153]:
abs(diff(cex_new(x,ve,jxbar,Cex,Ceystar,sigmastar,theta),x).subs(x,pe)) * pe

0.190750535457404

In [178]:
def cex_new(pe,ve, jxbar, Cex, Ceystar, sigmastar, theta):
    jxbar_p = Cex / (Cex + (g(ve) / g(pe)) ** theta * Ceystar)
    return Dstar(ve, sigmastar) / Dstar(1, sigmastar) * (jxbar_p / jxbar) ** (1+(1-sigmastar) / theta) * Cex

def ceystar_new(pe,ve, jxbar, Cex, Ceystar, sigmastar, theta):
    jxbar_p = Cex / (Cex + (g(ve) / g(pe)) ** theta * Ceystar)
    return Dstar(pe, sigmastar) / Dstar(1, sigmastar) * ((1-jxbar_p) / (1-jxbar)) ** (1+(1-sigmastar) / theta) * Ceystar

def jxbar_temp(pe,ve, theta, Cex, Ceystar):
    return Cex / (Cex + (g(ve) / g(pe)) ** theta * Ceystar)



In [191]:
diff(ceystar_new(x,x + tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe) * pe

-27.1319284807435

In [193]:
diff(ceystar_new(x,x+tb - prop*tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe) * pe + diff(cex_new(x,x+tb - prop*tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe) * pe * prop

-27.1884985168510

In [189]:
diff(cex_new(x,x+tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe) * pe * prop

-0.0657892368424103

In [195]:
diff(jxbar_temp(x,x+tb-prop*tb,theta,Cex,Ceystar),x).subs(x,pe)

0.00151264869451942

In [204]:
jxbar_prime * theta * (1-jxbar_prime) * g(pe) / g(ve)  * (gprime(ve) * g(pe) - gprime(pe) * g(ve)) / g(pe)**2

-0.0015126485460821854

In [209]:
dcexdpe 

0.310413131492890

In [210]:
diff(cex_new(x,x+tb - prop*tb, jxbar, Cex, Ceystar, sigmastar, theta),x).subs(x,pe)

-0.241822095519461

In [211]:
Dstarprime(ve, sigmastar) / Dstar(ve, sigmastar)

-0.856165439487672

In [212]:
(1+(1-sigmastar) / theta) / (jxbar_prime) * djxbardpe

0.106341184807621